# 7 Linear Regression with TensorFlow using the Boston Housing Dataset

The goal of this exercise is to implement a linear regression model using TensorFlow to predict house prices based on the California Housing Dataset. The dataset contains various features such as average income, housing average age, and more. Your task is to build a linear regression model and evaluate its performance.

Import the required libraries:

In [6]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import fetch_california_housing

Load the California Housing Dataset

In [10]:
raw = fetch_california_housing()
X = pd.DataFrame(data=raw['data'], columns=raw['feature_names'])
y = pd.Series(raw['target'])

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  
0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Length: 20640, dtype: float64


Preprocess the data:
* Normalize the features using the mean and standard deviation.
* Split the dataset into training and testing sets (e.g., 80% for training, 20% for testing).

Define the TensorFlow graph:
* Create placeholders for the input features (X) and target variable (y).
* Create variables for the model's weights (W) and bias (b).
* Define the linear regression model using the equation: y_pred = X * W + b.
* Define the loss function as the mean squared error between the predicted values and the true values.
* Choose an optimizer (e.g., Gradient Descent) to minimize the loss function.

Train the model:
* Initialize TensorFlow session.
* Initialize the model's variables.
* Set the number of training epochs and the learning rate.
* For each epoch, iterate through the training dataset and update the model's parameters using the optimizer.
* Print the training loss at regular intervals.

Evaluate the model:
* Use the trained model to make predictions on the test dataset.
* Calculate the mean squared error (MSE) between the predicted and true values.
* Print the MSE as a measure of the model's performance.